In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as io:

    SETTING = yaml.load(io)

PATH = make_path_dict(SETTING)

In [ ]:
if SETTING["signal_type"] != "context":

    raise

In [ ]:
feature_x_sample_processed = pd.read_csv(
    PATH["feature_x_sample.processed.tsv"], sep="\t", index_col=0
)

feature_x_sample_processed.columns.name = SETTING["sample_alias"]

In [ ]:
for element_type, dataframe in (
    ("feature", feature_x_sample_processed),
    ("sample", feature_x_sample_processed.T),
):

    if element_type not in SETTING["context_element_types"]:

        continue

    element_x_fit_parameter = kraft.fit_skew_t_pdf_on_each_dataframe_row(
        dataframe,
        n_job=SETTING["n_job"],
        output_file_path=PATH[f"{element_type}_x_fit_parameter.tsv"],
    )

    element_x_fit_parameter = pd.read_csv(
        PATH[f"{element_type}_x_fit_parameter.tsv"], sep="\t", index_col=0
    )

    for parameter, parameter_values in element_x_fit_parameter.items():

        parameter_values_not_na_unique = parameter_values.dropna().unique()

        if parameter_values_not_na_unique.size == 1:

            print(f"{parameter} is all {parameter_values_not_na_unique[0]}.")

            continue

        title = f"Fit {SETTING[f'{element_type}_alias']}"

        kraft.plot_histogram((parameter_values,), title=title, xaxis_title=parameter)

        n_extreme_per_direction = 2

        extreme_elements = kraft.select_series_indices(
            parameter_values,
            "<>",
            n=n_extreme_per_direction,
            title={"text": title},
            xaxis={"title": "Rank"},
            yaxis={"title": parameter.title()},
        ).tolist()

        extreme_elements = (
            extreme_elements[:n_extreme_per_direction]
            + extreme_elements[-n_extreme_per_direction:]
        )

        kraft.plot_histogram(
            tuple(
                dataframe.loc[extreme_element] for extreme_element in extreme_elements
            ),
            title=f"{SETTING[f'{element_type}_alias']}s with Extreme {parameter}",
            xaxis_title=SETTING["feature_x_sample_value_name"],
        )

        for extreme_element in extreme_elements:

            n_data, location, scale, degree_of_freedom, shape = element_x_fit_parameter.loc[
                extreme_element,
                ["N Data", "Location", "Scale", "Degree of Freedom", "Shape"],
            ]

            kraft.plot_context(
                dataframe.loc[extreme_element],
                n_data=n_data,
                location=location,
                scale=scale,
                degree_of_freedom=degree_of_freedom,
                shape=shape,
                xaxis_title=SETTING["feature_x_sample_value_name"],
            )